In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('../15_modified_data/train_data.csv')
train.head()

,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,fare,tip,additional_charges,shared_trip_authorized,trips_pooled,date,...,wind_speed,pickup_community_name,dropoff_community_name,year,month,day,day_of_week,hour,weekend,nonzero_tip
0,2019-09-24 06:30:00,2019-09-24 06:45:00,802.0,6.183132,10.0,3,2.55,False,1,2019-09-24,...,210.27236,LINCOLN PARK,NEAR WEST SIDE,2019,September,24,Tuesday,6,0,1
1,2019-07-30 11:45:00,2019-07-30 12:30:00,2349.0,18.248731,35.0,0,8.11,False,1,2019-07-30,...,161.05968,OHARE,NEAR NORTH SIDE,2019,July,30,Tuesday,11,0,0
2,2018-11-17 01:15:00,2018-11-17 01:30:00,684.0,5.277118,10.0,0,2.50,False,1,2018-11-17,...,161.05968,NEAR NORTH SIDE,NORTH CENTER,2018,November,17,Saturday,1,1,0
3,2019-12-09 12:00:00,2019-12-09 12:15:00,698.0,3.079843,7.5,1,2.55,False,1,2019-12-09,...,279.61750,WEST TOWN,NEAR WEST SIDE,2019,December,9,Monday,12,0,1
4,2018-12-17 15:00:00,2018-12-17 15:00:00,812.0,2.996371,7.5,0,2.50,False,1,2018-12-17,...,161.05968,SOUTH SHORE,GREATER GRAND CROSSING,2018,December,17,Monday,15,0,0


In [3]:
y = train['nonzero_tip']
len(y)


83703

In [4]:
X = pd.get_dummies(train, columns = ['dropoff_community_name', 'pickup_community_name'])
X.head()
X.drop(['nonzero_tip', 'tip', 'trip_start_timestamp', 'trip_end_timestamp', 'date' ], axis=1, inplace=True)
month_map = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}
X['month_int'] = X['month'].map(month_map)
day_map = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}
X['day_ofweek_int'] = X['day_of_week'].map(day_map)
X['day_ofweek_int'].value_counts()
X.drop(['day_of_week', 'month' ], axis=1, inplace=True)

In [5]:
import imblearn
from imblearn.under_sampling import RandomUnderSampler
import collections
from imblearn.over_sampling import RandomOverSampler


In [6]:
ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(X, y)
print('Original dataset shape', collections.Counter(y))
print('Resample dataset shape', collections.Counter(y_ros))

Original dataset shape Counter({0: 68654, 1: 15049})
Resample dataset shape Counter({1: 68654, 0: 68654})


In [26]:
rus = RandomUnderSampler(random_state=42)
x_rus, y_rus = rus.fit_resample(X, y)
print('Original dataset shape', collections.Counter(y))
print('Resample dataset shape', collections.Counter(y_rus))

Original dataset shape Counter({0: 68654, 1: 15049})
Resample dataset shape Counter({0: 15049, 1: 15049})


In [15]:
len(X) + len(y)
68654 + 15049
# 68654 + 68654

83703

In [11]:
len(x_ros)
# len(y_ros)

137308

In [16]:
val = pd.read_csv('../15_modified_data/val_data.csv')
val.head()
val_y = val['nonzero_tip']
len(val_y)
val_X = pd.get_dummies(val, columns = ['dropoff_community_name', 'pickup_community_name'])
val_X.head()
val_X.drop(['nonzero_tip', 'tip', 'trip_start_timestamp', 'trip_end_timestamp', 'date' ], axis=1, inplace=True)
month_map = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}
val_X['month_int'] = val_X['month'].map(month_map)
day_map = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}
val_X['day_ofweek_int'] = val_X['day_of_week'].map(day_map)
val_X['day_ofweek_int'].value_counts()
val_X.drop(['day_of_week', 'month' ], axis=1, inplace=True)


In [19]:
x_ros.drop(['dropoff_community_name_RIVERDALE'], axis = 1, inplace=True)

In [28]:
# x_rus.drop(['dropoff_community_name_RIVERDALE'], axis = 1, inplace=True)

In [20]:
log1 = LogisticRegression()
log1.fit(x_ros, y_ros)
ypred1 = log1.predict(val_X)
ypred1

/Users/michellevan/opt/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([0, 1, 1, ..., 0, 0, 1])

In [29]:
log2 = LogisticRegression()
log2.fit(x_rus, y_rus)
ypred2 = log2.predict(val_X)
ypred2

/Users/michellevan/opt/miniconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([0, 1, 1, ..., 0, 0, 1])

In [31]:
pred_prob1 = log1.predict_proba(val_X)
pred_prob2 = log2.predict_proba(val_X)

In [35]:
pred_prob1

array([[0.54355128, 0.45644872],
       [0.4567143 , 0.5432857 ],
       [0.48383123, 0.51616877],
       ...,
       [0.61995469, 0.38004531],
       [0.66178313, 0.33821687],
       [0.47864428, 0.52135572]])

In [21]:
from sklearn.metrics import roc_curve


In [32]:
fpr1, tpr1, thresh1 = roc_curve(val_y, pred_prob1[:,1], pos_label=1)
fpr2, tpr2, thresh2 = roc_curve(val_y, pred_prob2[:,1], pos_label=1)

# roc curve for tpr = fpr 
random_probs = [0 for i in range(len(val_y))]
p_fpr, p_tpr, _ = roc_curve(val_y, random_probs, pos_label=1)

In [34]:
from sklearn.metrics import roc_auc_score

# auc scores
auc_score1 = roc_auc_score(val_y, pred_prob1[:,1])
auc_score2 = roc_auc_score(val_y, pred_prob2[:,1])

print(auc_score1, auc_score2)

0.608746037366666 0.602395400811911
